# MosquitoAlert - YoloV5 Baseline Submission

![](https://images.aicrowd.com/raw_images/challenges/social_media_image_file/1124/1c83e8678106d51ab51f.png)

This notebook will contains instructions and example submission with yolov5 trained on the dataset.

## Installations 🤖

1. `aicrowd-cli` for downloading challenge data and making submissions
2. `pyarrow` for saving to parquet for submissions

In [1]:
!pip install -U ultralytics tqdm opencv-python torchvision pandas Pillow gdown aicrowd-cli

     -------------------------------------- 629.1/629.1 kB 4.0 MB/s eta 0:00:00
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
     ---------------------------------------- 38.1/38.1 MB 8.4 MB/s eta 0:00:00
     ---------------------------------------- 10.8/10.8 MB 8.1 MB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 8.4 MB/s eta 0:00:00
  Using cached aicrowd_cli-0.1.15-py3-none-any.whl (51 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
  Using cached rich-10.16.2-py3-none-any.whl (214 kB)
  Using cached GitPython-3.1.18-py3-none-any.whl (170 kB)
     ---------------------------------------- 1.0/1.0 MB 9.5 MB/s eta 0:00:00
  Using cached semver-2.13.0-py2.py3-none-any.whl (12 kB)
  Using cached requests_toolbelt-0.10.1-py2.py3-none-any.whl (54 kB)
  Using cached gitdb-4.0.10-py3-none-any.whl (62 kB)
  Using cached commonmark-0.9.1-py2.py3-none-any.whl (51 kB)
  Using cached smmap-5.0.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.2.2 requires pyqt5<5.13, which is not installed.
spyder 5.2.2 requires pyqtwebengine<5.13, which is not installed.
jupyter-client 7.3.4 requires pyzmq>=23.0, but you have pyzmq 22.1.0 which is incompatible.
black 22.6.0 requires click>=8.0.0, but you have click 7.1.2 which is incompatible.


In [4]:
import torch
import os
from pathlib import Path
import cv2
from timeit import default_timer as timer
import pandas as pd
from tqdm.auto import tqdm
from PIL import Image

# Download Dataset and Models

In [3]:
!aicrowd login

Please login here: https://api.aicrowd.com/auth/72GKtGKTvb9NeJLLPvmHZQh0QVkRq-9KXSYxKIAMShI
API Key valid
Gitlab access token valid
Saved details successfully!


In [4]:
!aicrowd dataset download --challenge mosquitoalert-challenge-2023


sample_submission_phase1_v2.csv:   0%|          | 0.00/359k [00:00<?, ?B/s]
sample_submission_phase1_v2.csv: 100%|██████████| 359k/359k [00:01<00:00, 293kB/s]
sample_submission_phase1_v2.csv: 100%|██████████| 359k/359k [00:01<00:00, 293kB/s]

test_images_phase1.zip:   0%|          | 0.00/2.74G [00:00<?, ?B/s]
test_images_phase1.zip:   0%|          | 1.05M/2.74G [00:01<1:16:00, 601kB/s]
test_images_phase1.zip:   0%|          | 2.10M/2.74G [00:02<40:11, 1.14MB/s] 
test_images_phase1.zip:   0%|          | 3.15M/2.74G [00:02<27:00, 1.69MB/s]
test_images_phase1.zip:   0%|          | 4.19M/2.74G [00:02<18:15, 2.50MB/s]
test_images_phase1.zip:   0%|          | 5.24M/2.74G [00:02<14:40, 3.11MB/s]
test_images_phase1.zip:   0%|          | 7.34M/2.74G [00:02<08:55, 5.11MB/s]
test_images_phase1.zip:   0%|          | 8.39M/2.74G [00:03<11:49, 3.86MB/s]
test_images_phase1.zip:   0%|          | 10.5M/2.74G [00:03<08:11, 5.56MB/s]
test_images_phase1.zip:   0%|          | 12.6M/2.74G [00:03<07:13, 6.3

In [5]:
!unzip -qq test_images_phase1.zip -d test_images_phase1/

In [6]:
# Downloading the weight file at https://drive.google.com/file/d/1s7EoK3V9YTwQOxhuO1ERABRZ95atz8PS/view
!gdown "1s7EoK3V9YTwQOxhuO1ERABRZ95atz8PS"

Downloading...
From: https://drive.google.com/uc?id=1s7EoK3V9YTwQOxhuO1ERABRZ95atz8PS
To: c:\Users\KIIT\Documents\GitHub\MosquitoAlert---YoloV5\mosquitoalert-yolov5-baseline.pt

  0%|          | 0.00/14.4M [00:00<?, ?B/s]
  4%|▎         | 524k/14.4M [00:00<00:08, 1.72MB/s]
 11%|█         | 1.57M/14.4M [00:00<00:03, 4.10MB/s]
 22%|██▏       | 3.15M/14.4M [00:00<00:01, 6.92MB/s]
 33%|███▎      | 4.72M/14.4M [00:00<00:01, 8.53MB/s]
 44%|████▍     | 6.29M/14.4M [00:00<00:00, 9.76MB/s]
 55%|█████▍    | 7.86M/14.4M [00:00<00:00, 10.7MB/s]
 66%|██████▌   | 9.44M/14.4M [00:01<00:00, 11.2MB/s]
 77%|███████▋  | 11.0M/14.4M [00:01<00:00, 10.7MB/s]
 88%|████████▊ | 12.6M/14.4M [00:01<00:00, 10.4MB/s]
 99%|█████████▊| 14.2M/14.4M [00:01<00:00, 9.72MB/s]
100%|██████████| 14.4M/14.4M [00:01<00:00, 8.99MB/s]


In [1]:
TEST_DATA_DIR = 'test_images_phase1/'

In [2]:
def classify_image(image):
    image_information = {}
    result = yolov5_model(image)
    result_df = result.pandas().xyxy[0]
    if result_df.empty:
        print('No results from yolov5 model!')
    else:
        image_information = result_df.to_dict()
    return image_information

# getting mosquito_class name from predicted result
def extract_predicted_mosquito_class_name(extractedInformation):
    mosquito_class = ""
    if extractedInformation is not None:
        mosquito_class = str(extractedInformation.get("name").get(0))
    return mosquito_class

# getting mosquito_class number from predicted result
def extract_predicted_mosquito_class_number(extractedInformation):
    mosquito_class = ""
    if extractedInformation is not None:
        mosquito_class = str(extractedInformation.get("class").get(0))
    return mosquito_class

# getting mosquito_class confidence score from predicted result
def extract_predicted_mosquito_class_confidence(extractedInformation):
    mosquito_class = ""
    if extractedInformation is not None:
        mosquito_class = str(extractedInformation.get("confidence").get(0))
    return mosquito_class

# getting mosquito bounding box from predicted result
# it looks like the results are in different notation
# than the uoutput file
# Pascal VOC? (top, left, bottom, right)?
def extract_predicted_mosquito_bbox(extractedInformation):
    bbox = []
    if extractedInformation is not None:
        xmin = str(extractedInformation.get("xmin").get(0))
        ymin = str(extractedInformation.get("ymin").get(0))
        xmax = str(extractedInformation.get("xmax").get(0))
        ymax = str(extractedInformation.get("ymax").get(0))
        bbox = [xmin, ymin, xmax, ymax]
    return bbox

# find image id
def find_image_id(original_image):
    image_name = os.path.splitext(original_image)[0]
    return image_name

In [5]:
# path to dataset
root_images = os.path.join(TEST_DATA_DIR)

all_images = os.listdir(root_images)
print(f"Total images: {len(all_images)}")

class_labels = {
    "aegypti":      0,
    "albopictus":   1,
    "anopheles":    2,
    "culex":        3,
    "culiseta":     4,
    "japonicus/koreicus": 5
}

# counter for correctly recognized mosquito classes
counter = 0
labels_counter = 0
no_mosquito_detected = 0
rows = []

# yolov5 challenge trained baseline
trained_model_pretrained = "mosquitoalert-yolov5-baseline.pt"

# yolov5_model
# Colab gets an error on this -> Use Runtime -> Restart Session (The data will not be deleted)
yolov5_model = torch.hub.load('ultralytics/yolov5', 'custom', path = trained_model_pretrained, force_reload = True)

for original_image in tqdm(all_images):
    try:
        original_image_file = os.path.join(root_images, original_image)
        # checking if it is a file
        if os.path.isfile(original_image_file):
            # opening testing image
            # print(f'You are watching: {original_image}')
            # classifying image by yolov5 model
            predictedInformation = classify_image(original_image_file)
            mosquito_class_name_predicted = ""
            mosquito_class_number_predicted = ""
            mosquito_class_confidence = 0
            mosquito_class_bbox = [0, 0, 0, 0]

            if predictedInformation:
                mosquito_class_name_predicted = extract_predicted_mosquito_class_name(predictedInformation)
                mosquito_class_number_predicted = extract_predicted_mosquito_class_number(predictedInformation)
                mosquito_class_confidence = extract_predicted_mosquito_class_confidence(predictedInformation)
                mosquito_class_bbox = extract_predicted_mosquito_bbox(predictedInformation)
                counter += 1
            else:
                no_mosquito_detected += 1
            # print(f"Predicted mosquito class: {mosquito_class_name_predicted} with {float(mosquito_class_confidence):.2f} confidence score.")
            #  bbox = [xmin, ymin, xmax, ymax]
            # create row for csv file
            row = [original_image, mosquito_class_name_predicted, mosquito_class_number_predicted, mosquito_class_confidence,
                   mosquito_class_bbox[0], mosquito_class_bbox[1], mosquito_class_bbox[2], mosquito_class_bbox[3]]
            rows.append(row)
            labels_counter += 1
            # print(f"Finished file reading, file {original_image} read correctly!")
    except Exception as e:
        print(f'Unable to process file: {original_image}!')
        print(f'Exception: {e}!')

Total images: 2676


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\KIIT/.cache\torch\hub\master.zip
requirements: Ultralytics requirement C:\Users\KIIT\.cache\torch\hub\ultralytics_yolov5_master\requirements.txt not found, attempting AutoUpdate...
     -------------------------------------- 188.5/188.5 kB 1.6 MB/s eta 0:00:00
  Attempting uninstall: gitpython
    Found existing installation: GitPython 3.1.18
    Uninstalling GitPython-3.1.18:
      Successfully uninstalled GitPython-3.1.18

requirements: AutoUpdate success  5.3s, installed 1 package: C:\Users\KIIT\.cache\torch\hub\ultralytics_yolov5_master\requirements.txt
requirements:  Restart runtime or rerun command for updates to take effect

YOLOv5  2023-7-13 Python-3.9.13 torch-2.0.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7026307 parameters, 0 gradients
Adding AutoShape... 


  0%|          | 0/2676 [00:00<?, ?it/s]

No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!
No results from yolov5 model!


In [6]:
df = pd.DataFrame(rows, columns=["img_fName", "class_label", "predicted_class_number", "confidence_score", "bbx_xtl", "bbx_ytl", "bbx_xbr", "bbx_ybr"])
sub_df = df.drop(['predicted_class_number', 'confidence_score'], axis=1, inplace=False)
sub_df.head()

,img_fName,class_label,bbx_xtl,bbx_ytl,bbx_xbr,bbx_ybr
0,000653c4-e448-4334-88f6-d0504f6d0150.jpeg,albopictus,304.1554870605469,890.7817993164062,818.8773803710938,1371.757080078125
1,000b2828-69d4-44d5-8441-c8429f3b82dc.jpeg,albopictus,240.91102600097656,500.74285888671875,487.7963562011719,694.3878173828125
2,002ce967-1ad4-4b30-b008-4aa3a2cf1af3.jpeg,culex,539.69921875,632.9386596679688,858.35546875,946.9381713867188
3,004aeee9-6083-4ed0-a5cc-72fb4fccc3e7.jpeg,culex,950.2664794921875,1237.094482421875,1582.0760498046875,2063.00927734375
4,004efaf0-2f65-4b32-ad33-9072e8e60591.jpeg,culex,182.4885711669922,47.25125503540039,525.2783203125,420.83099365234375


In [7]:
sub_df.to_csv('submission_phase1.csv', index=False)

In [8]:
!aicrowd submission create -c mosquitoalert-challenge-2023 -f "submission_phase1.csv"

submission_phase1.csv ------------ 100.0% • 328.0/326.3  • 386.2 kB/s • 0:00:00
                                            KB                                 
                                             ┌─────────────────────────┐                                              
                                             │ Successfully submitted! │                                              
                                             └─────────────────────────┘                                              
                                                   Important links                                                    
┌──────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────┐
│  This submission │ https://www.aicrowd.com/challenges/mosquitoalert-challenge-2023/submissions/235959              │
│                  │                                                                                                 │
│  All 